In [96]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [261]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
token_index = -1
wrapper()

['W', 0, 'program']
program
['I', 0, 'body']
['R', 4, ';']
['W', 11, 'begin']
['I', 1, 'a']
operator
assignment
['W', 12, ':=']
['I', 2, 'b']
expression
heh
hah
['R', 4, ';']
huh
['W', 18, 'end.']
expression checked
Fatal: ";" expected 


In [183]:
def scan():
    global tokens_chain, token_index, nxtsymb
    token_index += 1
    if token_index < len(tokens_chain):
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [184]:
def antiscan():
    global tokens_chain, token_index, nxtsymb
    token_index -= 1
    if token_index >= 0:
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [185]:
def error(text):
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL)
#     ahah/0

In [186]:
def check(response):
    if len(response):
        error(response)
        return False
    else:
        return True

In [230]:
def wrapper():
    global nxtsymb
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()   
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    while check(operator()):
        scan()
        print('hl')
    if nxtsymb[2] != 'end.':
        return '"end." expected'
    

In [210]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [211]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expexted'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        scan()
    if nxtsymb[2] != ':':
        return '":" expected'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [212]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return '"[" expected'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval syntax'
            scan()
        if nxtsymb[2] != ']':
            return '"]" expected'
        scan()
        if nxtsymb[2] != 'of':
            return '"of" expected'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'variable type expected'
        return ''
    else:
        return 'array type expected'
        
            

In [213]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'constant expected'
    scan()
    if nxtsymb[2] != '..':
        return '".." expected'
    scan()
    if nxtsymb[0] != 'C':
        return 'constant expected'
    return ''

In [ ]:
def operators():
    global nxtsymb
    
    if not(operator):
        return 'operator expected'
    

In [260]:
def operator():
    global nxtsymb
    print(Fore.GREEN + 'operator' + Style.RESET_ALL)
    
    if nxtsymb[2] == ";":
        pass
    elif nxtsymb[0] == 'I':
        if not check(assignment()):
            return 'invalid assignment declaration'
        scan()
        print('expression checked')
        if nxtsymb[2] != ';':
            return '";" expected'
    return ''

In [235]:
def assignment():
    global nxtsymb
    print(Fore.GREEN + 'assignment' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ':=':
        return '":=" expected'
    scan()
    if not check(expression()):
        return 'invalid expression'
    return ''

In [254]:
def expression():
    print(Fore.GREEN + 'expression' + Style.RESET_ALL)
    global nxtsymb
    
    if not check(term()):
        return 'term expected'
    scan()
    while nxtsymb[2] == '+':
        scan()
        if not check(factor()):
            return 'factor expected'
    print('huh')
    return ''

In [255]:
def term():
    global nxtsymb
    
    if not check(factor()):
        return 'factor expected'
    while nxtsymb[2] == '*':
        scan()
        if not check(factor()):
            return 'factor expected'
    print('hah')
    return ''

In [256]:
def factor():
    global nxtsymb
    
    if nxtsymb[2] == '(':
        scan()
        if not check(expression()):
            return 'expression expected'
        if nxtsymb[2] != ')':
            return '")" expected'
    else:
        if nxtsymb[0] not in ['I', 'C']:
            return 'constant or identifier expected'
        print('heh')
    return ''